# Tutorial: Entendimiento de los datos RaSa

Continuando con el proyecto de consultoria de Wide World Importers, el primer paso para iniciar la comprensión de los datos es explorar y entender las fuentes de datos disponibles. Note que esto también nos ayuda a comprender mejor la organización.

## Configuración e importe de paquetes
Se utilizará el paquete de pandas profiling para apoyar el análisis estadístico, y se importan los paquetes de python
necesarios

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions
from pyspark.sql.types import StructType
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql.types import FloatType, StringType, IntegerType, DateType
from pyspark.sql.functions import udf, col, length, isnan, when, count
import pyspark.sql.functions as f
import os 
from datetime import datetime
from pyspark.sql import types as t
from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt
import numpy as np

C:\Users\estudiante\anaconda3\envs\Tutoriales\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Configuración del controlador e inicio de sesion Spark

In [2]:
path_jar_driver = 'C:\Program Files (x86)\MySQL\Connector J 8.0\mysql-connector-java-8.0.28.jar'

In [3]:
#Configuración de la sesión
conf=SparkConf() \
    .set('spark.driver.extraClassPath', path_jar_driver)

spark_context = SparkContext(conf=conf)
sql_context = SQLContext(spark_context)
spark = sql_context.sparkSession

C:\Users\estudiante\anaconda3\envs\Tutoriales\lib\site-packages\pyspark\sql\context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [4]:
# Si quiere practicar la conexion con el servidor de base de datos:
db_connection_string = 'jdbc:mysql://157.253.236.120:8080/RaSaTransaccional'
# El usuario es su estudiante _i asignado y su contraseña la encontrará en el archivo excel de Coursera 
db_user = 'Estudiante_22_202415'
db_psswd = 'Estudiante_201117125'

PATH='./'

## Perfilamiento de los datos
El perfilamiento se basa en varios aspectos: **entendimiento general** de los datos compartidos, validación de **reglas de negocio**, **análisis descriptivo** utilizando estadística descriptiva y diagramas para la visualización de los datos y **análisis de calidad**. El primero hace referencia a tener una comprensiòn global antes de entrar en los detalles, la segunda sobre las reglas, corresponden a especificaciones dadas por el negocio en relación con los datos, un ejemplo de regla de negocio dada es: "Todo producto de la canasta familiar debe tener un precio de IVA equivalente a 19%", dada esta regla de negocio, se debe revisar en los datos suministrados que no hayan valores de IVA menores a 19%. En el tercer aspecto, relacionado con el análisis descriptivo, se revisan estadísticas: mínimo, máximo, media, mediana, distribuciones de los datos, valores nulos, cardinalidad con el fin de comprender en mayor nivel de profundidad los datos compartidos.


### Información dada por la organización relacionada con los datos
RaSA de forma conjunta con un grupo de consultores de inteligencia de negocios, ha trabajado en una especificación de los primeros análisis que quiere que usted realice. En esta primera fase, a partir de estos análisis propuestos, la empresa le ha entregado una serie de fuentes de datos y requiere que usted realice los siguientes entregables relacionados con la etapa de entendimiento de datos:

Dichos datos pueden tener errores ya que no han sido utilizados previamente para ser analizados. La empresa nos da información adicional sobre los datos:

- Las áreas de servicios reportadas cubren todos los condados del país
- Los tipos de beneficios con límite cuantitativo deben tener una cantidad límite diferente de cero en los planes que los ofrecen.
- Las fuentes FuenteAreasDeServicio_Copia_E y FuenteTiposBeneficio_Copia_E comparten información de los años 2017 al 2019
- La empresa comparte 5409 áreas de servicios y 170 tipos de beneficios.
- El valor máximo Copago y Coseguro para el año 2018 es respectivamente 3300 y 100.
- Además, les comparte información de 301 planes para 2017 y de 422 para el año 2018.
- Existen 15 y 5 diferentes condiciones de copago y coseguro respectivamente.


### Conexión a fuente de datos y acceso a los datos

#### Conexión a fuente de datos
A continuación encuentra las funciones para conectarse a la fuente de datos (archivo csv o base de datos) y retornar un dataframe que es el que se utilizará posteriormente para manipular los datos.

In [5]:
def obtener_dataframe_de_bd(db_connection_string, sql, db_user, db_psswd):
    df_bd = spark.read.format('jdbc')\
        .option('url', db_connection_string) \
        .option('dbtable', sql) \
        .option('user', db_user) \
        .option('password', db_psswd) \
        .option('driver', 'com.mysql.cj.jdbc.Driver') \
        .load()
    return df_bd

### Entendimiento general de datos "Fuente Condiciones de Pago"

#### Cargue de datos
Para consultar desde la base de datos se puede acceder directamente a la tabla dado su nombre o se puede especificar la sentencia SQL de un "SELECT" completo para traer la información total o parcial de la tabla que se requiere. A continuación un ejemplo de cada uno de estos usos. 
Acceso directo para el caso de órdenes y acceso con sentencia SQL para el caso de detallesOrdenes.

In [6]:
sql_areas = 'FuenteAreasDeServicio_Copia_E'


In [7]:
#Se cargan los dataframes desde la base de datos
areas = obtener_dataframe_de_bd(db_connection_string, sql_areas, db_user, db_psswd)


### Entendimiento general de datos "Fuente Condiciones de Pago"

Para ver una muestra de los datos, utilice el comando <i>show</i> ingresando el número de filas que quiere ver

In [8]:
areas.show(5)

+------------------+--------------------+-------------+-----------------+----------+------------+-----+--------+-----+
|IdAreaDeServicio_T|NombreAreaDeServicio|IdGeografia_T|          Condado|    Estado|PoblacionAct| Area|Densidad|Fecha|
+------------------+--------------------+-------------+-----------------+----------+------------+-----+--------+-----+
|         100622017|New Jersey - Medi...|        34005|Burlington County|New Jersey|    464269.0|805.0|   577.0| 2017|
|         101012018|New Jersey - Medi...|        34031|   Passaic County|New Jersey|    518117.0|185.0|  2801.0| 2018|
|          10132017|BlueOptions16842F...|        12031|     Duval County|   Florida|    999935.0|774.0|  1292.0| 2017|
|         101982018|New Jersey - Medi...|        34003|    Bergen County|New Jersey|    953819.0|234.0|  4076.0| 2018|
|         102012017|New Jersey - Medi...|        34021|    Mercer County|New Jersey|    385898.0|226.0|  1708.0| 2017|
+------------------+--------------------+-------

Como puede observar 

In [9]:
areas.schema

StructType(List(StructField(IdAreaDeServicio_T,IntegerType,true),StructField(NombreAreaDeServicio,StringType,true),StructField(IdGeografia_T,IntegerType,true),StructField(Condado,StringType,true),StructField(Estado,StringType,true),StructField(PoblacionAct,DoubleType,true),StructField(Area,DoubleType,true),StructField(Densidad,DoubleType,true),StructField(Fecha,IntegerType,true)))

Por otra parte, en lo

Una vez entendida en términos generales la estructura de los datos, es necesario conocer la cantidad de datos disponibles, esto se hace con count() y con columns, que retornan el número de filas y número de columnas respectivamente.

In [10]:
print((areas.count(), len(areas.columns)))

(188815, 9)


**¿Cuantos datos nos compartieron para el proyecto?**
En total tenemos 188815 instancias de Fuentes de areas de servicio en un total de 9 variables a revisar de las cuales 1 es un ID que identifica el tipo de servicio y 8 variables que muestran información sobre la ubicación de las áreas y fechas.

### Revisión de reglas de negocio 

Primero se confirman las reglas proporcionadas por el negocio, la primera que asegura que existen información desde 2017 al 2019

In [11]:
areas.agg({"Fecha": "max"}).show()
areas.agg({"Fecha": "min"}).show()

+----------+
|max(Fecha)|
+----------+
|      2018|
+----------+

+----------+
|min(Fecha)|
+----------+
|      1800|
+----------+



Se observa que el periodo de años de la tabla de Areas de Servicio está entre 1800 y 2018 - No corresponde a lo definido por el cliente

La siguiente regla asegura que las ***areas de servicio se distribuyen por todo el país***. Para este caso:

In [12]:
estados = areas.select("Estado").distinct()
num_estados = estados.count()
print(f"El número de estados donde se encuentran las áreas de servicio son: {num_estados}")
estados.show(35)

El número de estados donde se encuentran las áreas de servicio son: 35
+--------------+
|        Estado|
+--------------+
|        Hawaii|
|          Ohio|
|        Oregon|
|      Arkansas|
|         Texas|
|  North Dakota|
|  Pennsylvania|
|      Illinois|
|      Oklahoma|
|      Delaware|
|        Alaska|
|    New Mexico|
| West Virginia|
|      Missouri|
|       Georgia|
|       Montana|
|      Virginia|
|      Michigan|
|North Carolina|
|       Wyoming|
|        Kansas|
|    New Jersey|
|       Alabama|
|       Arizona|
|      Kentucky|
|     Louisiana|
|   Mississippi|
| New Hampshire|
|     Tennessee|
|       Florida|
|       Indiana|
|South Carolina|
|  South Dakota|
|     Wisconsin|
|         Maine|
+--------------+



In [13]:
condados = areas.select("Condado").distinct()
num_condados = condados.count()
print(f"El número de condados donde se encuentran las áreas de servicio son: {num_condados}")
estados.show(10)

El número de condados donde se encuentran las áreas de servicio son: 1398
+------------+
|      Estado|
+------------+
|      Hawaii|
|        Ohio|
|      Oregon|
|    Arkansas|
|       Texas|
|North Dakota|
|Pennsylvania|
|    Illinois|
|    Oklahoma|
|    Delaware|
+------------+
only showing top 10 rows



Como resultado se observa que se definen áreas de servicio solo para ***35 estados*** y ***1398*** condados

In [14]:
descripcion = areas.select("Condado", "Estado").distinct().groupBy("Condado").count()
descripcion.show()

+--------------------+-----+
|             Condado|count|
+--------------------+-----+
|         Owen County|    2|
|   Williamson County|    3|
|Lewis and Clark C...|    1|
|     Canadian County|    1|
|     McLennan County|    1|
|         Rock County|    1|
|       Unicoi County|    1|
|     Woodward County|    1|
|   Cumberland County|    8|
|      Webster County|    5|
|     Muskegon County|    1|
|    Hempstead County|    1|
|      Wexford County|    1|
|      Kingman County|    1|
|         Cole County|    1|
|      Rosebud County|    1|
|       DuPage County|    1|
|      El Paso County|    1|
|    Robertson County|    2|
|     Montcalm County|    1|
+--------------------+-----+
only showing top 20 rows



##### Duplicados en Condados:

Se observa que algunos condados aparecen múltiples veces con diferentes estados, lo cual puede ser inconsistente ya que un condado generalmente pertenece a un solo estado. Por ejemplo:

Owen County aparece 2 veces.

Williamson County aparece 3 veces.

Cumberland County aparece 8 veces.

Webster County aparece 5 veces.

##### Inconsistencias en los Datos:

La presencia de múltiples estados asociados a un mismo condado puede indicar un error de datos, posiblemente debido a:

Duplicación de datos.

Errores en la entrada de datos.

Asociación incorrecta de estados a condados.

### Análisis descriptivo
Una vez cubierto lo básico de reglas de negocio, pasamos a perfilamiento estadístico. Para ver estadisticos generales como conteo, mínimo, máximo, desviación estandar y percentiles utilice summary()

In [15]:
areas.summary().show()

+-------+--------------------+--------------------+------------------+----------------+-------+-------------------+------------------+-----------------+------------------+
|summary|  IdAreaDeServicio_T|NombreAreaDeServicio|     IdGeografia_T|         Condado| Estado|       PoblacionAct|              Area|         Densidad|             Fecha|
+-------+--------------------+--------------------+------------------+----------------+-------+-------------------+------------------+-----------------+------------------+
|  count|              182527|              188815|            182437|          188815| 188815|             188815|            186408|           186408|            188815|
|   mean|5.5661602010935366E7|                null| 42946.06266820875|            null|   null|4.147959051463602E7| 770.9518851122269|276.0979410754903|2010.2215554908244|
| stddev|3.0402410466496132E7|                null|32489.478070072095|            null|   null|5.943835931334065E8|1263.4887340485252|851.32

In [16]:
poblacion = [row['PoblacionAct'] for row in areas.select('PoblacionAct').collect()]
densidad = [row['Densidad'] for row in areas.select('Densidad').collect()]

plt.figure(figsize=(10, 6))
plt.scatter(poblacion, densidad, color='green', alpha=0.5)
plt.title('Relación entre Población Activa y Densidad')
plt.xlabel('Población Activa')
plt.ylabel('Densidad')
plt.show()


C:\Users\estudiante\anaconda3\envs\Tutoriales\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  if __name__ == '__main__':


Concentración de Datos:

La mayoría de los puntos de datos están concentrados cerca del origen (valores bajos tanto en población activa como en densidad), lo que indica que hay muchas áreas con baja población activa y baja densidad.

Distribución de Datos:

A medida que se incrementan los valores de población activa, la densidad también tiende a incrementarse, aunque en menor cantidad.

Hay algunos puntos de datos aislados (outliers) en la parte superior del gráfico, que representan áreas con densidades muy altas en relación con la población activa.

In [17]:
import seaborn as sns
estado_counts = areas.groupBy("Estado").count().toPandas()
plt.figure(figsize=(12, 6))
sns.barplot(x='Estado', y='count', data=estado_counts, palette='viridis')
plt.title('Cantidad de Registros por Estado')
plt.xlabel('Estado')
plt.ylabel('Cantidad de Registros')
plt.xticks(rotation=45)
plt.show()


C:\Users\estudiante\anaconda3\envs\Tutoriales\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  if __name__ == '__main__':


Estados con mayor cantidad de registros:

Texas, Virginia y South Carolina destacan con más de 12,000 registros cada uno, situándose como los estados con la mayor cantidad de registros.

Otros estados como Pennsylvania, Oklahoma, Georgia, y Florida también tienen una cantidad significativa de registros, superando los 8,000.

Estados con menor cantidad de registros:

Hay estados que muestran una cantidad menor de registros en comparación, lo que puede reflejar variaciones en la población, la actividad económica o la implementación de políticas públicas que afectan los registros en esos estados.

###### Conclusion del analisis descriptivo 

El análisis revela que los datos presentan una amplia variabilidad en términos de población activa, área y densidad. La alta desviación estándar en varias columnas sugiere una gran diversidad en los datos, lo cual es importante para la toma de decisiones y la planificación estratégica. Sin embargo, algunas estadísticas parecen no tener sentido, especialmente considerando la cantidad de estados y condados en Estados Unidos. Por ejemplo:

La PoblacionAct presenta un valor máximo extremadamente alto (47,280,300,000), lo cual no es coherente con la población real de ningún estado o condado en Estados Unidos.

El Área tiene un valor mínimo negativo (-24,707), lo cual es imposible ya que el área no puede ser negativa.

Estas inconsistencias indican la necesidad de una revisión y corrección de los datos para asegurar la precisión y relevancia de los análisis futuros. Identificar y corregir estos outliers es crucial para mantener la integridad del conjunto de datos y tomar decisiones informadas.

## Análisis de calidad de datos
La calidad de los datos consiste en validar la idoneidad de los datos analizando varias dimensiones, entre las cuales resaltamos:
- Completitud: Datos que no existen o no se conocen
- Unicidad (Duplicidad): Datos que son idénticos en diferentes registros o registros con valores idénticos en atributos en los que no debería ocurrir
- Consistencia: Esta medida se define por la validez o integridad del dato, en la definición de su estructura, al interior de una fila o entre diferentes filas de la misma fuente o de diferentes fuentes. El manejo de unidades en los datos y el significado del dato son generadores de inconsistencias.
- Validez: a nivel de formato y de sentido de los datos más alla de sus valores

Para asegurar una calidad mínima de los datos en estas dimensiones hay que entender su origen y posteriormente realizar un proceso de validación de cada una de las dimensiones de perfilamiento que llevará a un entendimiento de los datos disponibles.


A continuación se definen dos funciones, la primera cuenta los valores vacios por columna de dataframe, la segunda retorna un diccionario de columnas con cardinalidad superior al 50%

Te invito a revisar los recursos de calidad de datos que hemos visto en el curso, a continuacion ejemplos daremos algunos ejemplos por cada dimensión para el caso del tutorial

### Unicidad y validez

La unicidad se revisó en la segunda regla de negocio y la validez se reviso en el perfilamiento general, en la regla de negocio 3 y en el análisis de la descripción al final de la sección de revisión de reglas de negocio

### Completitud y validez

In [26]:
# Validar completitud en la tabla areas
areas_faltantes = areas.select([col(c).isNull().alias(c) for c in areas.columns]).groupBy().sum()
areas_faltantes.show()


++
||
++
||
++



In [28]:
# Contar duplicados en la tabla areas
areas_duplicados = areas.groupBy(areas.columns).count().filter("count > 1")
numero_duplicados = areas_duplicados.count()
print(f"Cantidad de registros duplicados: {numero_duplicados}")


Cantidad de registros duplicados: 43573


La tabla presenta 43,573 registros duplicados y 0 problemas de comletitud.

Identificar estos registros duplicados es crucial para asegurar la integridad y precisión de los datos.

### Consistencia

In [24]:
from pyspark.sql import functions as F

duplicados_id = areas.groupBy("IdAreaDeServicio_T").count().filter("count > 1")
if duplicados_id.count() == 0:
    print("No hay duplicados en IdAreaDeServicio_T.")
else:
    print("Hay duplicados en IdAreaDeServicio_T:")
    duplicados_id.show()
duplicados_id.count()

Hay duplicados en IdAreaDeServicio_T:
+------------------+-----+
|IdAreaDeServicio_T|count|
+------------------+-----+
|          35462018|   79|
|          10222017|   87|
|         111392017|    8|
|          24642018|   56|
|          85952018|   85|
|          48692018|   35|
|          99002017|   73|
|          75512018|   62|
|           2942018|   42|
|           1482017|   21|
|          38822017|   10|
|          39412017|   14|
|         109842017|    9|
|         111512017|   14|
|          56982018|   14|
|          57862018|   10|
|         109402017|   14|
|          33932017|   12|
|         108582018|   15|
|          40312018|    5|
+------------------+-----+
only showing top 20 rows



5384

In [25]:
# Completitud de NombreAreaDeServicio
areas.select([F.count(F.when(F.col("NombreAreaDeServicio").isNull(), 1)).alias("NombreAreaDeServicio_nulos")]).show()


+--------------------------+
|NombreAreaDeServicio_nulos|
+--------------------------+
|                         0|
+--------------------------+



In [ ]:
# Duplicados de IdGeografia_T en combinación con Condado y Estado
duplicados_geografia = areas.groupBy("IdGeografia_T", "Condado", "Estado").count().filter("count > 1")
if duplicados_geografia.count() == 0:
    print("IdGeografia_T es único por Condado y Estado.")
else:
    print("Hay duplicados en IdGeografia_T para ciertas combinaciones de Condado y Estado:")
    duplicados_geografia.show()
duplicados_geografia.count()

In [ ]:
areas.filter(F.col("PoblacionAct") <= 0).show()

In [ ]:
a = areas.filter(F.col("Area") <= 0).show()
count_area_inusual = areas.filter(F.col("Area") <= 0).count()

print(f"Cantidad de registros con valores negativos en 'Area': {count_area_inusual}")

In [ ]:
from datetime import datetime

año_actual = datetime.now().year
areas.filter((F.col("Fecha") < 1700) | (F.col("Fecha") > año_actual)).show()


##### Unicidad y validez:

La unicidad fue revisada en la segunda regla de negocio y se verificó que existen 5384 duplicados en la columna IdAreaDeServicio_T. La validez fue comprobada mediante el perfilamiento general y las reglas de negocio 3, de tal manera que los datos no cumplen con las condiciones esperadas.

##### Completitud y validez:

Se identificó que la columna Tipo presenta 2 valores faltantes (NaN), lo que podría requerir una intervención para garantizar la calidad del dataset.
En cuanto a la cardinalidad, ninguna de las columnas tiene más del 50% de valores distintos en comparación con el total de registros, lo que sugiere que las columnas tienen una variedad razonable de valores sin ser excesivamente heterogéneas.

##### Consistencia:

Se encontraron duplicados en la columna IdAreaDeServicio_T. Algunos valores, como el IdAreaDeServicio_T con valor 35462018, se repiten 79 veces. Esto indica un posible error en el registro de datos o una inconsistencia en la asignación de identificadores, lo que debe ser revisado y corregido.
No se encontraron valores nulos en la columna NombreAreaDeServicio, lo que asegura que la información está completa en esta área.
Se identificaron duplicados en la combinación de IdGeografia_T, Condado y Estado, lo que sugiere que para ciertas combinaciones geográficas hay registros repetidos. Este aspecto también debe ser revisado para garantizar la unicidad en la base de datos.

##### Valores negativos o inusuales:

Se encontraron 6177 registros con valores negativos en la columna Area, lo cual es una inconsistencia significativa, ya que no es lógico que el área de una geografía tenga un valor negativo. Esto debe ser investigado más a fondo para determinar si es un error de entrada de datos o si tiene algún significado específico que deba ser considerado.
En cuanto a la columna PoblacionAct, no se encontraron registros con valores negativos, lo que es una señal positiva de consistencia en esa variable.

##### Fechas en un rango razonable:

Las fechas en la columna Fecha se encuentran dentro del rango esperado. No se encontraron fechas que estuvieran fuera del intervalo entre el año 1700 y el año actual, se encontró que el valor mínimo es 1800, lo que indica que las fechas están correctamente formateadas y son razonables.